# Time-Domain OCT

Time-Domain OCT was the first implementation of OCT and provides the most intuitive understanding of how OCT works.

## Learning Objectives

By the end of this notebook, you will understand:
1. How TD-OCT systems work
2. Mechanical reference arm scanning
3. A-scan and B-scan generation
4. Limitations of TD-OCT
5. Why Fourier-Domain OCT was developed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. TD-OCT Principle

In Time-Domain OCT:
1. The reference mirror is **mechanically scanned** in depth
2. At each position, interference is measured
3. Interference only occurs when path lengths match (within coherence length)
4. The envelope of the interference signal reveals tissue structure

### Key Components:
- Low-coherence light source
- Beam splitter
- Sample arm (fixed)
- Reference arm (scanning mirror)
- Photodetector

In [ ]:
def td_oct_signal(z_ref, sample_structure, wavelength=840e-9, coherence_length=7e-6):
    """
    Simulate TD-OCT signal acquisition.
    
    Parameters:
    - z_ref: Reference mirror positions (array)
    - sample_structure: List of (depth, reflectivity) tuples
    - wavelength: Center wavelength (m)
    - coherence_length: Coherence length (m)
    """
    signal = np.ones_like(z_ref)  # DC component
    
    for depth, reflectivity in sample_structure:
        # Path length difference
        delta_z = z_ref - depth
        
        # Coherence envelope
        envelope = np.exp(-(delta_z / coherence_length)**2)
        
        # Interference fringes
        carrier = np.cos(4 * np.pi * delta_z / wavelength)
        
        # Add contribution from this reflector
        signal += reflectivity * envelope * carrier
    
    return signal

# Define sample structure (e.g., layers of tissue)
sample = [
    (50e-6, 0.7),   # Surface (50 μm, 70% reflectivity)
    (120e-6, 0.5),  # Layer 1
    (200e-6, 0.4),  # Layer 2
    (280e-6, 0.3),  # Layer 3
]

# Reference mirror scan range
z_ref = np.linspace(0, 350e-6, 3000)

# Acquire signal
signal = td_oct_signal(z_ref, sample)

# Plot raw interferogram
plt.figure(figsize=(14, 6))
plt.plot(z_ref * 1e6, signal, 'b', linewidth=1)
plt.xlabel('Reference Mirror Position (μm)', fontsize=12)
plt.ylabel('Photodetector Signal (a.u.)', fontsize=12)
plt.title('TD-OCT Raw Interferogram', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Mark sample positions
for depth, refl in sample:
    plt.axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("This is what the photodetector sees as the reference mirror scans!")

## 2. Extracting the A-Scan

To recover tissue structure, we extract the envelope using demodulation.

In [ ]:
# Demodulate to get A-scan
signal_ac = signal - np.mean(signal)
analytic_signal = hilbert(signal_ac)
a_scan = np.abs(analytic_signal)

# Convert to dB scale (typical in OCT)
a_scan_db = 20 * np.log10(a_scan / np.max(a_scan))

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Linear scale
ax1.plot(z_ref * 1e6, a_scan, 'g', linewidth=2)
ax1.set_xlabel('Depth (μm)', fontsize=12)
ax1.set_ylabel('Reflectivity (linear)', fontsize=12)
ax1.set_title('TD-OCT A-Scan (Linear Scale)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
for depth, refl in sample:
    ax1.axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5)

# dB scale (standard in OCT)
ax2.plot(z_ref * 1e6, a_scan_db, 'b', linewidth=2)
ax2.set_xlabel('Depth (μm)', fontsize=12)
ax2.set_ylabel('Reflectivity (dB)', fontsize=12)
ax2.set_title('TD-OCT A-Scan (Logarithmic Scale)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-40, 0)
for depth, refl in sample:
    ax2.axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("dB scale is used because OCT has very large dynamic range (40-50 dB)")

## 3. Building a B-Scan (2D Image)

A B-scan is created by:
1. Acquiring multiple A-scans at different lateral positions
2. Stacking them side-by-side
3. Displaying as a 2D image

In [ ]:
def generate_sample_structure(x_position, base_depth=100e-6, variation=20e-6):
    """
    Generate a sample with varying layer depths (simulate curved surface).
    """
    # Add some variation to layer depths based on x position
    variation_factor = np.sin(2 * np.pi * x_position / 200e-6)
    
    structure = [
        (50e-6 + variation * variation_factor, 0.7),
        (120e-6 + variation * variation_factor * 0.8, 0.5),
        (200e-6 + variation * variation_factor * 0.6, 0.4),
        (280e-6 + variation * variation_factor * 0.4, 0.3),
    ]
    
    return structure

# Generate B-scan
num_ascans = 200  # Number of lateral positions
x_positions = np.linspace(0, 400e-6, num_ascans)
z_positions = np.linspace(0, 350e-6, 500)

b_scan = np.zeros((len(z_positions), num_ascans))

print("Acquiring B-scan... (simulating lateral scanning)")
for i, x_pos in enumerate(x_positions):
    # Generate sample structure for this lateral position
    sample_at_x = generate_sample_structure(x_pos)
    
    # Acquire A-scan
    signal = td_oct_signal(z_positions, sample_at_x)
    signal_ac = signal - np.mean(signal)
    a_scan = np.abs(hilbert(signal_ac))
    
    # Store in B-scan
    b_scan[:, i] = a_scan
    
    if (i + 1) % 50 == 0:
        print(f"  Acquired {i+1}/{num_ascans} A-scans")

print("B-scan complete!")

# Convert to dB
b_scan_db = 20 * np.log10(b_scan / np.max(b_scan))
b_scan_db = np.clip(b_scan_db, -40, 0)

# Display B-scan
plt.figure(figsize=(14, 8))
plt.imshow(b_scan_db, cmap='gray', aspect='auto', 
           extent=[0, x_positions[-1]*1e6, z_positions[-1]*1e6, 0])
plt.colorbar(label='Reflectivity (dB)', shrink=0.8)
plt.xlabel('Lateral Position (μm)', fontsize=12)
plt.ylabel('Depth (μm)', fontsize=12)
plt.title('TD-OCT B-Scan (Cross-Sectional Image)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("This is a 2D cross-sectional image showing tissue layers!")

## 4. TD-OCT Limitations

### Speed
- Mechanical scanning is **slow**
- Typical: 100-400 A-scans/second
- Limited by mirror inertia and settling time

### Motion Artifacts
- Slow acquisition → sensitive to sample motion
- Problem for in-vivo imaging

### Sensitivity
- Limited by detector noise
- Typically 80-100 dB

In [ ]:
# Simulate acquisition speed comparison
td_oct_speed = 400  # A-scans per second
fd_oct_speed = 40000  # A-scans per second (Fourier-Domain)

# For a typical B-scan
num_ascans_per_bscan = 500

td_time = num_ascans_per_bscan / td_oct_speed
fd_time = num_ascans_per_bscan / fd_oct_speed

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

methods = ['Time-Domain\nOCT', 'Fourier-Domain\nOCT']
times = [td_time, fd_time]
colors = ['red', 'green']

bars = ax.barh(methods, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels
for bar, time in zip(bars, times):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2, 
            f'{time:.3f} s', ha='left', va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('Time to Acquire 500 A-scans (seconds)', fontsize=12)
ax.set_title('Acquisition Speed Comparison', fontsize=14, fontweight='bold')
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

speedup = td_time / fd_time
print(f"TD-OCT: {td_time:.2f} seconds per B-scan")
print(f"FD-OCT: {fd_time:.3f} seconds per B-scan")
print(f"\nFourier-Domain OCT is {speedup:.0f}x faster!")
print("\nThis speed advantage:")
print("  ✓ Reduces motion artifacts")
print("  ✓ Enables 3D imaging")
print("  ✓ Improves patient comfort")

## 5. Real-World TD-OCT Applications

Despite limitations, TD-OCT was revolutionary:

### Historical Impact
- First OCT images of retina (1991)
- FDA approval for ophthalmic imaging (1996)
- Foundation for modern OCT technology

### Current Use
- Mostly replaced by FD-OCT in clinical settings
- Still used in some research applications
- Important for understanding OCT principles

## Summary

In this notebook, we've learned:

1. ✅ **TD-OCT mechanism**: Mechanical scanning of reference mirror

2. ✅ **A-scan generation**: Demodulation extracts tissue structure

3. ✅ **B-scan creation**: Multiple A-scans form 2D images

4. ✅ **Limitations**: Speed, motion sensitivity, sensitivity

5. ✅ **Historical importance**: Foundation of OCT technology

## Key Points

**Advantages:**
- Simple and intuitive
- Direct depth scanning
- Well-understood technology

**Disadvantages:**
- Slow acquisition (limited by mechanics)
- Motion artifacts
- Lower sensitivity than FD-OCT

## Next Steps

In the next notebook, we'll explore:
- Fourier-Domain OCT (FD-OCT)
- How spectral analysis eliminates mechanical scanning
- Why FD-OCT is 50-100x faster
- Spectral-Domain vs. Swept-Source OCT

---

**Continue to Notebook 5: Fourier-Domain OCT →**